In [1]:
import os
import numpy as np
import SimpleITK as sitk
from pathlib import Path

# Mapping des labels FreeSurfer
LABELS_MAP = {
    0: "Background", 2: "Left-Cerebral-White-Matter", 3: "Left-Cerebral-Cortex",
    4: "Left-Lateral-Ventricle", 5: "Left-Inf-Lat-Vent", 7: "Left-Cerebellum-White-Matter",
    8: "Left-Cerebellum-Cortex", 10: "Left-Thalamus-Proper", 11: "Left-Caudate",
    12: "Left-Putamen", 13: "Left-Pallidum", 14: "3rd-Ventricle", 15: "4th-Ventricle",
    16: "Brain-Stem", 17: "Left-Hippocampus", 18: "Left-Amygdala", 24: "CSF",
    26: "Left-Accumbens-area", 28: "Left-VentralDC", 30: "Left-vessel", 31: "Left-choroid-plexus",
    41: "Right-Cerebral-White-Matter", 42: "Right-Cerebral-Cortex", 43: "Right-Lateral-Ventricle",
    44: "Right-Inf-Lat-Vent", 46: "Right-Cerebellum-White-Matter", 47: "Right-Cerebellum-Cortex",
    49: "Right-Thalamus-Proper", 50: "Right-Caudate", 51: "Right-Putamen", 52: "Right-Pallidum",
    53: "Right-Hippocampus", 54: "Right-Amygdala", 58: "Right-Accumbens-area",
    60: "Right-VentralDC", 62: "Right-vessel", 63: "Right-choroid-plexus",
    72: "5th-Ventricle", 77: "WM-hypointensities", 80: "non-WM-hypointensities", 85: "Optic-Chiasm",
    251: "CC_Posterior", 252: "CC_Mid_Posterior", 253: "CC_Central", 254: "CC_Mid_Anterior", 255: "CC_Anterior",
}

# --- Setup des chemins ---
PATIENT, TP = "P1", "T1"
MODALITIES = ["T1", "T2", "FLAIR"]

base_path = Path(f"/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/27919209/MSLesSegDataset/train/{PATIENT}/{TP}")
proj_path = Path("/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project")

aseg_path = proj_path / "data/07_registered_aseg_results" / f"{PATIENT}_{TP}_aseg.nii"
out_dir = proj_path / "data/16_template_txt"
out_dir.mkdir(parents=True, exist_ok=True)

def compute_2d_stats(img_slice, mask_slice, l_id):
    """Calcule la morphologie et le contraste sur la coupe cible."""
    l_zone = (mask_slice == l_id).astype(np.uint8)
    if not np.any(l_zone): return None

    # Intensités
    vals = img_slice[l_zone == 1]
    stats = {"mean": vals.mean(), "min": vals.min(), "max": vals.max()}
    
    # Géométrie via SITK
    img_sitk = sitk.GetImageFromArray(l_zone)
    shape = sitk.LabelShapeStatisticsImageFilter()
    shape.Execute(img_sitk)
    
    if shape.HasLabel(1):
        dims = shape.GetEquivalentEllipsoidDiameter(1)
        stats["major"], stats["minor"] = max(dims), min(dims)
    else: return None

    # Contraste simple (moyenne lésion / moyenne reste de la coupe)
    bg_mean = img_slice[l_zone == 0].mean()
    stats["contrast"] = stats["mean"] / bg_mean if bg_mean > 0 else 1.0
    return stats

def process_patient():
    mask_file = base_path / f"{PATIENT}_{TP}_MASK.nii.gz"
    if not mask_file.exists():
        print(f"Error: Mask missing -> {mask_file}")
        return

    # Chargement volumes
    m_img = sitk.ReadImage(str(mask_file))
    m_labels = sitk.RelabelComponent(sitk.ConnectedComponent(m_img > 0))
    m_arr = sitk.GetArrayFromImage(m_labels)
    n_lesions = int(m_arr.max())
    
    aseg_arr = sitk.GetArrayFromImage(sitk.ReadImage(str(aseg_path))) if aseg_path.exists() else None
    if aseg_arr is None: print("Warning: No ASEG found, location will be empty.")

    print(f"Processing {PATIENT} - {TP}: {n_lesions} lesions found.")

    for l_id in range(1, n_lesions + 1):
        # On cherche la slice avec le plus de pixels de la lésion l_id
        l_mask = (m_arr == l_id)
        best_z = np.argmax(l_mask.sum(axis=(1, 2)))
        
        data = {}
        for mod in MODALITIES:
            p = base_path / f"{PATIENT}_{TP}_{mod}.nii.gz"
            if p.exists():
                img = sitk.GetArrayFromImage(sitk.ReadImage(str(p)))
                res = compute_2d_stats(img[best_z], m_arr[best_z], l_id)
                if res: data[mod] = res
            
        if not data: continue

        # On prend FLAIR par défaut pour la morpho
        ref = data.get("FLAIR", list(data.values())[0])
        
        # Identification anatomique
        loc_str = "N/A"
        if aseg_arr is not None:
            l_pixels = aseg_arr[best_z][m_arr[best_z] == l_id]
            ids, counts = np.unique(l_pixels, return_counts=True)
            loc_list = []
            for i, c in zip(ids, counts):
                name = LABELS_MAP.get(int(i), f"ID-{int(i)}")
                loc_list.append(f"{name} ({100*c/len(l_pixels):.1f}%)")
            loc_str = " | ".join(loc_list)

        # Build du rapport texte
        report = [
            f"ID: {PATIENT}_{TP}_L{l_id}",
            f"Slice index (axial): {best_z}",
            "-"*30,
            f"Geometry: {ref['major']:.2f} x {ref['minor']:.2f} mm",
            "Intensities & Contrast:"
        ]
        for mod, s in data.items():
            report.append(f"  {mod:5}: Moy={s['mean']:.1f}, Contrast={s['contrast']:.2f}")
        
        report.append(f"\nLocation:\n  {loc_str}")

        out_name = f"{PATIENT}_{TP}_L{l_id}_desc.txt"
        with open(out_dir / out_name, "w", encoding="utf-8") as f:
            f.write("\n".join(report))
            
    print(f"Reports saved in {out_dir}")

if __name__ == "__main__":
    process_patient()
    
    # Check rapide du dernier fichier
    print("\n--- Quick Check ---")
    txt_files = list(out_dir.glob("*.txt"))
    if txt_files:
        with open(txt_files[-1], "r") as f:
            print(f.read())

Processing P1 - T1: 18 lesions found.
Reports saved in /Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project/data/16_template_txt

--- Quick Check ---
RAPPORT MÉDICAL: Patient P1 | T1 | Lésion 5
COUPE AXIALE: 76
MORPHOLOGIE (2D):
 - Grand axe: 23.50 mm
 - Petit axe: 4.60 mm

SIGNAL & CONTRASTE:
 - T1: Moy=144.8, Contrast=2.09
 - T2: Moy=280.4, Contrast=1.93
 - FLAIR: Moy=204.2, Contrast=2.91

LOCALISATION ANATOMIQUE:
 - Right-Cerebral-White-Matter (89.4%) | Right-Cerebral-Cortex (7.1%) | WM-hypointensities (3.5%)



In [6]:
import numpy as np
import SimpleITK as sitk
from pathlib import Path

# =================================================================
# 1. MATHÉMATIQUES TCHEBICHEF 3D
# =================================================================

def tchebichef_polynomials(N, order):
    M = np.zeros((order, N))
    x = np.arange(N)
    M[0, :] = 1.0
    if order > 1: M[1, :] = (2.0 * x - N + 1.0) / N
    for p in range(2, order):
        M[p, :] = ((2.0*p-1.0)*(2.0*x-N+1.0)/(p*N))*M[p-1,:] - ((p-1.0)/p)*(1.0-((p-1.0)**2/(N**2)))*M[p-2,:]
    return M

def calculate_rho(N, order):
    rho = np.zeros(order); rho[0] = N
    for p in range(1, order): rho[p] = (N**2-p**2)/(N**2) * (2*p-1)/(2*p+1) * rho[p-1]
    return rho

def decompose_3d(volume, order):
    D, H, W = volume.shape
    od, oh, ow = min(order, D), min(order, H), min(order, W)
    Md, Mh, Mw = tchebichef_polynomials(D, od), tchebichef_polynomials(H, oh), tchebichef_polynomials(W, ow)
    rd, rh, rw = calculate_rho(D, od), calculate_rho(H, oh), calculate_rho(W, ow)
    
    T_raw = np.einsum('zyx,pz,qy,rx->pqr', volume, Md, Mh, Mw)
    norm = 1.0 / np.einsum('p,q,r->pqr', rd, rh, rw)
    
    T_final = np.zeros((order, order, order))
    T_final[:od, :oh, :ow] = T_raw * norm
    return T_final

# =================================================================
# 2. CONFIGURATION ET LABELS
# =================================================================

LABELS_MAP = {
    0: "Background", 2: "Left-Cerebral-White-Matter", 3: "Left-Cerebral-Cortex",
    4: "Left-Lateral-Ventricle", 7: "Left-Cerebellum-White-Matter", 8: "Left-Cerebellum-Cortex",
    16: "Brain-Stem", 17: "Left-Hippocampus", 24: "CSF", 41: "Right-Cerebral-White-Matter",
    42: "Right-Cerebral-Cortex", 43: "Right-Lateral-Ventricle", 251: "CC_Posterior", 255: "CC_Anterior"
}

PATIENT, TP = "P1", "T1"
MODALITIES = ["T1", "T2", "FLAIR"]
ORDRE_TCHEBY = 10

# Définition des chemins avec pathlib.Path
base_path = Path(f"/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/27919209/MSLesSegDataset/train/{PATIENT}/{TP}")
proj_path = Path("/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project")

aseg_path = proj_path / "data" / "07_registered_aseg_results" / f"{PATIENT}_{TP}_aseg.nii"
out_dir = proj_path / "data" / "17_automated_reports"

# Création du dossier de sortie
out_dir.mkdir(parents=True, exist_ok=True)

# =================================================================
# 3. PIPELINE AUTOMATISÉ
# =================================================================

def run_global_analysis():
    print(f"🚀 Démarrage de l'analyse : {PATIENT} {TP}")
    
    # 1. Chargement du Masque
    mask_file = base_path / f"{PATIENT}_{TP}_MASK.nii.gz"
    if not mask_file.exists():
        print(f"❌ Erreur : Masque introuvable à {mask_file}")
        return

    m_img = sitk.ReadImage(str(mask_file))
    m_labels = sitk.RelabelComponent(sitk.ConnectedComponent(m_img > 0))
    m_arr = sitk.GetArrayFromImage(m_labels)
    n_lesions = int(m_arr.max())
    
    # 2. Chargement de l'Atlas Anatomique (ASEG)
    aseg_arr = None
    if aseg_path.exists():
        aseg_arr = sitk.GetArrayFromImage(sitk.ReadImage(str(aseg_path)))
    
    # 3. Chargement des Volumes IRM (Dictionnaire de tableaux numpy)
    volumes_irm = {}
    for mod in MODALITIES:
        p = base_path / f"{PATIENT}_{TP}_{mod}.nii.gz"
        if p.exists():
            volumes_irm[mod] = sitk.GetArrayFromImage(sitk.ReadImage(str(p)))

    # 4. Boucle sur chaque lésion individuelle
    for l_id in range(1, n_lesions + 1):
        print(f"   Analyse Lésion {l_id}/{n_lesions}...")
        
        # --- A. Découpe de la ROI 3D pour Tchebichef ---
        indices = np.where(m_arr == l_id)
        z_min, z_max = indices[0].min(), indices[0].max()
        y_min, y_max = indices[1].min(), indices[1].max()
        x_min, x_max = indices[2].min(), indices[2].max()
        
        # Extraction du volume (on utilise FLAIR pour la signature de forme)
        roi_vol = volumes_irm["FLAIR"][z_min:z_max+1, y_min:y_max+1, x_min:x_max+1]
        roi_mask = (m_arr[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1] == l_id)
        
        # Normalisation pour Tchebichef
        roi_final = roi_vol * roi_mask
        roi_norm = roi_final / (np.max(roi_final) + 1e-10)
        
        # Calcul des moments 3D
        moments_3d = decompose_3d(roi_norm, ORDRE_TCHEBY)
        
        # --- B. Analyse sur la Coupe de Référence (Best Slice) ---
        best_z_local = np.argmax(roi_mask.sum(axis=(1, 2)))
        best_z_global = z_min + best_z_local
        
        # Intensités et Contraste
        signal_stats = {}
        for mod, vol in volumes_irm.items():
            pixels_lesion = vol[best_z_global][m_arr[best_z_global] == l_id]
            mean_val = np.mean(pixels_lesion)
            # Moyenne du cerveau sain sur la même coupe
            bg_val = np.mean(vol[best_z_global][m_arr[best_z_global] == 0])
            signal_stats[mod] = {
                "moy": mean_val, 
                "contrast": mean_val / bg_val if bg_val > 0 else 1.0
            }

        # Localisation Anatomique (Recouvrement)
        loc_str = "Atlas non disponible"
        if aseg_arr is not None:
            pixels_aseg = aseg_arr[best_z_global][m_arr[best_z_global] == l_id]
            ids, counts = np.unique(pixels_aseg, return_counts=True)
            parts = [f"{LABELS_MAP.get(int(i), f'ID-{i}')} ({100*c/len(pixels_aseg):.1f}%)" for i, c in zip(ids, counts)]
            loc_str = " | ".join(parts)

        # --- C. Écriture du Rapport ---
        report_lines = [
            f"RAPPORT AUTOMATISÉ : {PATIENT}_{TP}_L{l_id}",
            f"Localisation (Z-Global): {best_z_global}",
            f"Dimensions ROI (px): {roi_norm.shape}",
            "-"*40,
            "1. MOMENTS DE TCHEBICHEF (3D):",
            f"   Ordre: {ORDRE_TCHEBY}",
            f"   Moment d'énergie (T000): {moments_3d[0,0,0]:.4f}",
            "",
            "2. CARACTÉRISTIQUES DU SIGNAL:",
        ]
        for mod, s in signal_stats.items():
            report_lines.append(f"   {mod:5}: Moy={s['moy']:.1f}, Contraste={s['contrast']:.2f}")
            
        report_lines.append(f"\n3. ANATOMIE:\n   {loc_str}")

        # Sauvegarde du fichier texte
        txt_name = f"{PATIENT}_{TP}_L{l_id}_full_analysis.txt"
        with open(out_dir / txt_name, "w", encoding="utf-8") as f:
            f.write("\n".join(report_lines))
            
        # Sauvegarde de la signature numérique pour l'IA (format binaire .npy)
        np.save(out_dir / f"{PATIENT}_{TP}_L{l_id}_moments.npy", moments_3d)

    print(f"✅ Analyse terminée. Dossier : {out_dir}")

if __name__ == "__main__":
    run_global_analysis()

🚀 Démarrage de l'analyse : P1 T1
   Analyse Lésion 1/18...
   Analyse Lésion 2/18...
   Analyse Lésion 3/18...
   Analyse Lésion 4/18...
   Analyse Lésion 5/18...
   Analyse Lésion 6/18...
   Analyse Lésion 7/18...
   Analyse Lésion 8/18...
   Analyse Lésion 9/18...
   Analyse Lésion 10/18...
   Analyse Lésion 11/18...
   Analyse Lésion 12/18...
   Analyse Lésion 13/18...
   Analyse Lésion 14/18...
   Analyse Lésion 15/18...
   Analyse Lésion 16/18...
   Analyse Lésion 17/18...
   Analyse Lésion 18/18...
✅ Analyse terminée. Dossier : /Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project/data/17_automated_reports


In [5]:
import os
PROJET_ROOT = "/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project"
ASEG_DIR = os.path.join(PROJET_ROOT, "data", "07_registered_aseg_results")

if os.path.exists(ASEG_DIR):
    files = os.listdir(ASEG_DIR)
    print(f"✅ Dossier trouvé. Il contient {len(files)} fichiers.")
    print("Voici les 10 premiers fichiers :")
    for f in files[:10]:
        print(f" - {f}")
else:
    print(f"❌ LE DOSSIER N'EXISTE PAS : {ASEG_DIR}")

✅ Dossier trouvé. Il contient 1 fichiers.
Voici les 10 premiers fichiers :
 - P1_T1_aseg.nii
